In [3]:
import sys
import os
import pandas as pd

sys.path.insert(0, "{}\\OPT SQL Queries".format(os.path.abspath('..')))

from SQL_Commands import connect_to_opt, merge_job_num_and_phase


import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)

import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package genesis to
[nltk_data]     C:\Users\cameron.bell\AppData\Roaming\nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cameron.bell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cameron.bell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cameron.bell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [10]:
import sys
import os
import pandas as pd
from tqdm import tqdm
sys.path.insert(0, "{}\\OPT SQL Queries".format(os.path.abspath('..')))

from SQL_Commands import connect_to_opt, merge_job_num_and_phase


connection = connect_to_opt()

sql_query = '''SELECT        dbo.tPrjCTREntity.id, dbo.tPrjCTREntity.display AS text
FROM            dbo.tPrjCTR INNER JOIN
                         dbo.tPrjCTREntity ON dbo.tPrjCTR.id = dbo.tPrjCTREntity.CTR'''

dataset = pd.read_sql_query(sql_query,connection).set_index('id')

connection = connect_to_opt()

sql_query = '''SELECT        dbo.tPrjCTREntity.id, dbo.tSysDepartment.display AS Department, dbo.tPrjCTREntityRoleHistory.hours
FROM            dbo.tPrjCTR INNER JOIN
                         dbo.tPrjCTREntity ON dbo.tPrjCTR.id = dbo.tPrjCTREntity.CTR INNER JOIN
                         dbo.tPrjCTREntityRole ON dbo.tPrjCTREntity.id = dbo.tPrjCTREntityRole.CTREntity INNER JOIN
                         dbo.tGenRole ON dbo.tPrjCTREntityRole.role = dbo.tGenRole.id INNER JOIN
                         dbo.tSysDepartment ON dbo.tGenRole.department = dbo.tSysDepartment.id INNER JOIN
                         dbo.tPrjCTREntityRoleHistory ON dbo.tPrjCTREntityRole.id = dbo.tPrjCTREntityRoleHistory.CTREntityRole
GROUP BY dbo.tPrjCTREntity.id, dbo.tSysDepartment.display, dbo.tPrjCTREntityRoleHistory.hours
ORDER BY dbo.tPrjCTREntityRoleHistory.hours'''

departments = pd.read_sql_query(sql_query,connection)

connection = connect_to_opt()

sql_query = '''SELECT        dbo.tPrjCTREntity.id, dbo.tGenClient.code AS Client, dbo.tPrjCTREntity.display AS [Activty/Deliverable]
FROM            dbo.tPrjCTR INNER JOIN
                         dbo.tPrjCTREntity ON dbo.tPrjCTR.id = dbo.tPrjCTREntity.CTR INNER JOIN
                         dbo.tPrjProject ON dbo.tPrjCTR.project = dbo.tPrjProject.id INNER JOIN
                         dbo.tGenClient ON dbo.tPrjProject.client = dbo.tGenClient.id'''

client_tasks = pd.read_sql_query(sql_query,connection)

In [26]:
client_tasks.loc[(client_tasks['Activty/Deliverable'].str.lower().str.contains('drawing'))|
                 (client_tasks['Activty/Deliverable'].str.lower().str.contains('dwg'))].drop_duplicates(subset=['Activty/Deliverable']).sort_values('Activty/Deliverable').iloc[50:100]

,id,Client,Activty/Deliverable
19769,24945,BOG,Ana platform approach drawing
21231,26574,TAL,Approve 34No. Drawings
37250,45463,JGC,Architectural Drawings
24557,30557,JGC,Architectural Drawings (Concept)
24808,30845,JGC,Architectural Drawings (Detailed)
24370,30345,JGC,Architectural Drawings - Concept
26659,33092,JGC,Architectural Drawings - Control
24817,30854,JGC,Architectural Drawings - Detail
5972,7689,DAN,As Built Drawings
30940,38372,SEP,As built drawings from initial Site Survey


In [30]:
length = len(client_tasks.drop_duplicates(subset=['Activty/Deliverable']))*10

from datetime import timedelta

print(timedelta(seconds=length))

2 days, 16:38:30


In [13]:
example = client_tasks.loc[client_tasks['Client']=='TAQ']
example['Activty/Deliverable'].value_counts()#.iloc[50:100]

PM: Project Controls & DCC                     80
PM: Project Management & Engineering           79
INST: Instrument Engineering                   79
PIP: Isometrics Rev C                          79
PRO: Process Engineering                       61
                                               ..
Valve & Material Selection                      1
7.2 TS: HSE Input/RA of Ex. Gel Pig Removal     1
ORBIT use                                       1
TS: Data Gathering IDC & Client Liaison         1
Removal Procedure (Rev O1)                      1
Name: Activty/Deliverable, Length: 2142, dtype: int64

In [7]:
dataset.drop_duplicates(subset='text',inplace=True)

for id in tqdm(dataset.index):
    df = departments.loc[departments['id']==id]
    if df.empty == False:
        dataset.loc[id,'Department'] = df['Department'].iloc[0]

100%|██████████| 23256/23256 [01:12<00:00, 321.68it/s]


In [21]:
from superintendent.distributed import ClassLabeller
from IPython.display import display, Markdown

def display_text(data):
    display(Markdown(data.loc['text']),Markdown(data.loc['Department']))

widget = ClassLabeller(
    connection_string="sqlite:///activities.db",
    display_func = display_text
)
widget

ClassLabeller(children=(HBox(children=(FloatProgress(value=0.0, description='Progress:', max=1.0),)), Box(chil…

In [10]:
widget.add_features(dataset)

In [23]:
widget.features

AttributeError: 'ClassLabeller' object has no attribute 'features'

In [33]:
class KNN_NLC_Classifer():
    def __init__(self, k=1, distance_type = 'path'):
        self.k = k
        self.distance_type = distance_type

    # This function is used for training
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    # This function runs the K(1) nearest neighbour algorithm and
    # returns the label with closest match. 
    def predict(self, x_test):
        self.x_test = x_test
        y_predict = []

        for i in tqdm(range(len(x_test))):
            max_sim = 0
            max_index = 0
            for j in range(self.x_train.shape[0]):
                temp = self.document_similarity(x_test[i], self.x_train[j])
                if temp > max_sim:
                    max_sim = temp
                    max_index = j
            y_predict.append(self.y_train[max_index])
        return y_predict

    def convert_tag(self, tag):
        """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
        tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
        try:
            return tag_dict[tag[0]]
        except KeyError:
            return None


    def doc_to_synsets(self, doc):
        """
            Returns a list of synsets in document.
            Tokenizes and tags the words in the document doc.
            Then finds the first synset for each word/tag combination.
        If a synset is not found for that combination it is skipped.

        Args:
            doc: string to be converted

        Returns:
            list of synsets
        """
        tokens = word_tokenize(doc+' ')
        
        l = []
        tags = nltk.pos_tag([tokens[0] + ' ']) if len(tokens) == 1 else nltk.pos_tag(tokens)
        
        for token, tag in zip(tokens, tags):
            syntag = self.convert_tag(tag[1])
            syns = wn.synsets(token, syntag)
            if (len(syns) > 0):
                l.append(syns[0])
        return l

    
    def similarity_score(self, s1, s2, distance_type = 'path'):
        """
        Calculate the normalized similarity score of s1 onto s2
        For each synset in s1, finds the synset in s2 with the largest similarity value.
        Sum of all of the largest similarity values and normalize this value by dividing it by the
        number of largest similarity values found.

        Args:
            s1, s2: list of synsets from doc_to_synsets

        Returns:
            normalized similarity score of s1 onto s2
        """
        s1_largest_scores = []

        for i, s1_synset in enumerate(s1, 0):
            max_score = 0
            for s2_synset in s2:
                if distance_type == 'path':
                    score = s1_synset.path_similarity(s2_synset, simulate_root = False)
                else:
                    score = s1_synset.wup_similarity(s2_synset)                  
                
                if score != None:
                    if score > max_score:
                        max_score = score
            
            if max_score != 0:
                s1_largest_scores.append(max_score)
        
        mean_score = np.mean(s1_largest_scores)
                
        return mean_score

    def document_similarity(self,doc1, doc2):
          """Finds the symmetrical similarity between doc1 and doc2"""

          synsets1 = self.doc_to_synsets(doc1)
          synsets2 = self.doc_to_synsets(doc2)
          
          return (self.similarity_score(synsets1, synsets2) + self.similarity_score(synsets2, synsets1)) / 2


In [28]:
doc1 = dataset.loc[47156,'text']
doc2 = dataset.loc[47157,'text']
x = KNN_NLC_Classifer()
print("Test Similarity Score: ", x.document_similarity(doc1, doc2))

Test Similarity Score:  0.271780303030303


In [30]:
import re
from tqdm import tqdm 
nltk.download('stopwords')
s = stopwords.words('english')
#add additional stop words
s.extend(['today', 'tomorrow', 'outside', 'out', 'there'])
ps = nltk.wordnet.WordNetLemmatizer()
for i in tqdm(dataset.index):
    review = re.sub('[^a-zA-Z]', ' ', dataset.loc[i,'text'])
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in s]
    review = ' '.join(review)
    dataset.loc[i, 'text'] = review

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cameron.bell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 38843/38843 [00:40<00:00, 951.14it/s]


In [141]:
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(stop_words="english",ngram_range=(1,3))



departments = dataset['Department'].drop_duplicates()
df_lst = []
for department in tqdm(departments):
    df = dataset.loc[dataset['Department']==department].copy()
    if df.empty == False:
        vec.fit(df['text'].values)
        features = vec.transform(df['text'].values)
        if len(df) >120:
            n_clusters = 30
        else:
            n_clusters = int(len(df)/4)
        if n_clusters > 0: 
            clust = KMeans(init='k-means++',n_clusters=n_clusters, n_init=10)
            clust.fit(features)
            df.loc[:,'Cluster Labels'] = clust.labels_
            df_lst.append(df)

100%|██████████| 22/22 [00:55<00:00,  2.52s/it]


In [ ]:
clust.

In [176]:
import random

sample_percentage = 0.1

dic = dict()
for df in tqdm(df_lst):
    n_clusters = df['Cluster Labels'].max()
    department = df.iloc[0,1]
    dic[department] = []
    
    for i in range(n_clusters):
        cluster_sample = df.loc[df['Cluster Labels']==i]
        if cluster_sample.empty == False:
            sample_size = int(len(cluster_sample)*sample_percentage)
            if sample_size < 1:
                sample_index = 0
                sample = cluster_sample.iloc[sample_index,0]
                dic[department].append(sample)
            else:
                sample_index = random.sample(range(0, len(cluster_sample)-1), sample_size)
                samples = list(cluster_sample.iloc[sample_index,0])
                for sample in samples:
                    dic[department].append(sample)



dic

100%|██████████| 18/18 [00:00<00:00, 52.02it/s]


ening',
  'Structural engineering ',
  'Engineering Support Stools',
  'Architectural Engineering ',
  'Port side LQ engineering ',
  'Structural Engineering Phase ',
  'Engineering for Replacement Barge',
  'DETAIL DESIGN AND ENGINEERING',
  'Structural engineering review',
  'Structural Engineering Phase 2',
  'Engineering Scope ',
  'Engineering design report',
  '02-PS-797 - Engineering',
  'Pre work Engineering',
  'Structural Engineering ',
  'Option 2  Engineering ',
  'Strucutral Engineering',
  'Elmagco brake additional assesment ',
  'Detail Design Additional Scope ',
  'Stage 4 - Reporting',
  'Reporting of strength results',
  'Trial tower additional pipe supports - Design',
  'Analysis and reporting',
  'Additional Scopes',
  'Calculation & Reporting',
  'Analysis including reporting',
  'Reporting and Plot plan updates ',
  'Project additional scope November 2015',
  'Reporting < Conclusions ',
  'Structural Modelling (Concept)',
  'Pipe Deck Concept ',
  'Report developm

In [173]:
random.sample(range(0, len(cluster_sample)-1), 1)

ValueError: Sample larger than population or is negative

In [187]:
department = 'Engineering (Structural)'
lst = dic[department]
print(len(lst))
lst

529


['PS-98411 pipe support repair',
 'Fabrication Support to Finnings',
 'Concpet design of support truss',
 'Anomaly beam 26 and pipe support 1.',
 'Support Stool Installation Procedure',
 'Structural Support Services ',
 'Structural IDC of Pipe Support Modifications',
 'Support / Lift Pin',
 'Support Frames',
 'Inspection & Release of Antenna Support',
 'Hours for construction support',
 'Pipe Support Detailed Designs',
 '3D modelling of support',
 'Under Deck Drain Pot Support Platform ',
 'Construction Support',
 '4.6 - STRUC: Mods at 4" Line head to support Equip',
 'Complete SD Pipe support drawings ',
 'Architectural Drawings - Control',
 'O1 DRAWINGS + CHECKING',
 'As Built Drawings',
 'Installation Drawings C1',
 'Detail design drawings ',
 'Structural drawings',
 'Site preparation drawings',
 'Clump weight fabircation drawings',
 'Detail Design Drawings ',
 'Steelwork Drawings',
 'Access Hatch Design Drawings',
 'Approve 34No. Drawings',
 'Structual Drawings ',
 'PP Jacket Prima

In [188]:
list(departments).index(department)

6

In [191]:
i = 6
j = 25

df_lst[i].loc[df_lst[i]['Cluster Labels']==j]

,text,Department,Cluster Labels
id,,,
2010,Technical & Material Specifications,Engineering (Structural),25
5955,Panel Procurement,Engineering (Structural),25
8921,Material grades and module weights,Engineering (Structural),25
9573,Material Coordination,Engineering (Structural),25
10791,Procurement Deliverables,Engineering (Structural),25
14789,Material Model Development,Engineering (Structural),25
15408,Preparation of material requisition,Engineering (Structural),25
15996,Material logistics,Engineering (Structural),25
16004,Material logisitcs,Engineering (Structural),25


In [130]:
for df in tqdm(df_lst[:5]):
    n_clusters = df['Cluster Labels'].max()
    department = df.iloc[0,1]
    
    for i in range(n_clusters):
        cluster_sample = df.loc[df['Cluster Labels']==i]

    

100%|██████████| 5/5 [00:00<00:00, 41.68it/s]


In [133]:
len()

,text,Department,Cluster Labels
id,,,
19,Preliminary Plany layout & Hazerdous Areas Dia...,Engineering (Piping & Mechanical),2
24,Insulation Spection,Engineering (Piping & Mechanical),2
28,Equipment & Skid Layouts,Engineering (Piping & Mechanical),2
29,Piping specs,Engineering (Piping & Mechanical),2
30,Piping GA,Engineering (Piping & Mechanical),2
...,...,...,...
47116,3D layout of the new equipment,Engineering (Piping & Mechanical),2
47117,Valve List & Data Sheets,Engineering (Piping & Mechanical),2
47124,Weight/Dimensional data,Engineering (Piping & Mechanical),2


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(stop_words="english",ngram_range=(1,3))
vec.fit(dataset['text'].values)
features = vec.transform(dataset['text'].values)

In [37]:
clust.fit(features)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=30, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [38]:
yhat = clust.predict(features)

In [39]:
dataset['Cluster Labels'] = clust.labels_
dataset

,text,Cluster Labels
id,,
2,project management,17
3,basis design,12
4,key philosophy,22
5,flare blowdown study,22
6,fuel gas study,22
...,...,...
47157,lopa preparation,22
47158,lopa facilitation,8
47159,lopa report,22


In [58]:
for i in range(0,30):

    print(i,':',len(dataset.loc[dataset['Cluster Labels']==i]))

0 : 375
1 : 569
2 : 89
3 : 432
4 : 1032
5 : 389
6 : 406
7 : 1024
8 : 160
9 : 349
10 : 612
11 : 624
12 : 793
13 : 236
14 : 643
15 : 288
16 : 692
17 : 454
18 : 329
19 : 2265
20 : 257
21 : 600
22 : 23774
23 : 265
24 : 558
25 : 364
26 : 243
27 : 274
28 : 134
29 : 613
